# tkinter - Quick and Dirty GUIs with Python

### "But I don't know how to use the command line"

*- User*

## So what is tkinter anyway?

* Tk is a GUI library originally built for the Tcl language [tcl-lang.org](https://www.tcl-lang.org/)
* tkinter is a Python wrapper around the Tk GUI library
* tkinter has been in the Python standard library a long time
* Ttk is a newer set of themed widgets (accessible via the `tkinter.ttk` module)


## Installing

Even though tkinter is part of the Python standard library, your Linux distribution might not have the tkinter 

`sudo apt-get install python3-tk`

## Hello World

![hello_world_app](images/hello_world.png "Title")

In [1]:
# Hello World
from tkinter import Tk
from tkinter.ttk import Frame, Label, Button

root = Tk()  # create a root window
root.title('Hello World') # Set the title of the window
top_level = root.winfo_toplevel()
top_level.geometry('250x100')

# Frame is a rectangular area where components can be placed
frame = Frame(root, padding=10, width=400, height=400)  
frame.grid()  # use grid layout to position widgets inside frame

# Add a Label and Button
Label(frame, text="Hello World").grid(column=0, row=0)
Button(frame, text="Quit", command=root.destroy).grid(column=0, row=1)

root.mainloop()

## Some refactoring

In [2]:
# Encapsulate the boilerplate
def build_root_and_frame(title, win_geometry='200x200'):
    root1 = Tk()
    root1.title(title)
    top_level1 = root1.winfo_toplevel()
    top_level1.geometry(win_geometry)
    frame1 = Frame(root1, padding=10)
    frame1.grid()
    return root1, frame1

def grid(widget, row, column, row_span=1, column_span=1, **kwargs):
    widget.grid(row=row, column=column, rowspan=row_span, columnspan=column_span, **kwargs)

## Input/Output

![input_output](images/input_output.png "Title")

In [3]:
# Take some input: Hello NAME
from tkinter import StringVar
from tkinter.ttk import Entry

root, frame = build_root_and_frame('Hello NAME', '400x150')

# This is known as a control value it's the value underlying the widget
# also comes in DoubleVar and IntVar flavors
name_in = StringVar()
name_out = StringVar()

def greet(*args):
    name_out.set(f"Hello {name_in.get()}")

grid(Label(frame, text='Name'), 0, 0)
grid(Entry(frame, width=20, textvariable=name_in), 0, 1)
grid(Button(frame, text='Greet', command=greet), 0, 2)

grid(Label(frame, textvariable=name_out), 1, 0)
grid(Button(frame, text='Quit', command=root.destroy), 2, 0)

root.mainloop()

## Canvas

![canvas.png](images/canvas.png "Canvas")

In [4]:
# another factory function
def grid_label(parent, text, row, column):
    grid(Label(parent, text=text), row, column)

In [13]:
from tkinter import Canvas

root, frame = build_root_and_frame('Canvas')
canvas = Canvas(frame, height=100, width=100, background='white')

def draw():
    canvas.create_line(0, 50, 50, 0)
    canvas.create_arc(50, 0, 100, 50)
    canvas.create_oval(50, 50, 100, 100)
    canvas.create_rectangle(25, 75, 75, 25)

grid(canvas, 0, 0)
grid(Button(frame, text='draw', command=draw), 0, 1)

root.mainloop()

## Checkbutton

![checkbox.png](images/checkbox.png "checkbox")

In [14]:
from tkinter.ttk import Checkbutton

root, frame = build_root_and_frame('Checkbutton', '400x100')

hackme = StringVar()
grid(
    Checkbutton(
        frame,
        text="Save credit card info so it can be stolen later",
        variable=hackme,
        onvalue='Hack the Planet!',
        offvalue="Oh no you don't!",
    ), 
    0, 0,
)
grid(Label(frame, textvariable=hackme), 1, 0)

root.mainloop()

## Radiobutton

![radiobutton.png](images/radiobutton.png "radiobutton")

In [15]:
from tkinter.ttk import Radiobutton

root, frame = build_root_and_frame('Radiobutton')

language = StringVar()
grid(Radiobutton(frame, text="English", variable=language, value='EN'), 0, 0)
grid(Radiobutton(frame, text="Españole", variable=language, value='ES'), 1, 0)
grid_label(frame, 'Value', 2, 0)
grid(Label(frame, textvariable=language), 2, 1)

root.mainloop()

## Radiobutton, with Style

The diamond Radiobuttons are wierd, but you can switch your app to a different theme to avoid them

![radiobutton_with_style](images/radiobutton_with_style.png "radiobutton with style")

In [16]:
from tkinter.ttk import Style

style = Style()
print(f"{style.theme_names()=}")
print(f"{style.theme_use()=}")

style.theme_names()=('clam', 'alt', 'default', 'classic')
style.theme_use()='default'


In [18]:
root = Tk()
root.style = Style(root)
root.style.theme_use('alt')
root.title("Radiobutton, with style")

frame = Frame(root, padding=10)
frame.grid()

language = StringVar()
grid(Radiobutton(frame, text="English", variable=language, value='EN'), 0, 0, sticky='w')
grid(Radiobutton(frame, text="Español", variable=language, value='ES'), 1, 0, sticky='w')
grid_label(frame, 'Value', 2, 0)
grid(Label(frame, textvariable=language), 2, 1)

root.mainloop()

## Listbox

![Listbox](images/listbox.png "Listbox")

In [19]:
from string import ascii_letters
from tkinter import BROWSE, EXTENDED, Listbox, MULTIPLE, SINGLE, W
root, frame = build_root_and_frame('Listbox')

options = [
    'alpha', 'bravo', 'charlie', 'delta', 'echo',
    'foxtrot', 'golf', 'hotel', 'india', 'juliet',
    'kilo', 'lima', 'mike', 'november', 'oscar',
    ascii_letters * 5,
]
options_var = StringVar(value=options)
listbox = Listbox(frame, height=10, listvariable=options_var)
chosen_var = StringVar()
select_mode = StringVar()
select_mode.set(BROWSE)

def show_selection():
    selected_indices = listbox.curselection()
    selections = [listbox.get(i) for i in selected_indices]
    chosen_var.set("selection: " + ", ".join(selections))
    
def mode_changed():
    listbox.selection_clear(0)
    listbox.configure(selectmode=select_mode.get())
    
def build_radio(mode):
    return Radiobutton(frame, text=mode.upper(), variable=select_mode, value=mode, command=mode_changed)

grid(listbox, 0, 0, row_span=5)
grid(build_radio(BROWSE), 0, 1, sticky=W)
grid(build_radio(SINGLE), 1, 1, sticky=W)
grid(build_radio(MULTIPLE), 2, 1, sticky=W)
grid(build_radio(EXTENDED), 3, 1, sticky=W)
grid(Button(frame, text='Read Selection', command=show_selection), 4, 1)
grid(Label(frame, textvariable=chosen_var), 5, 0, column_span=2)

root.mainloop()

## Scrollbar

![Scrolling Listbox](images/scrolling_listbox.png "Scrolling Listbox")

In [11]:
from tkinter import E, HORIZONTAL, N, S, VERTICAL
from tkinter.ttk import Scrollbar

root, frame = build_root_and_frame('Scrolling Listbox')
x_scroll = Scrollbar(frame, orient=HORIZONTAL)
y_scroll = Scrollbar(frame, orient=VERTICAL)

grid(x_scroll, 1, 0, sticky=E+W)
grid(y_scroll, 0, 1, sticky=N+S)

options_var = StringVar(value=options)
listbox = Listbox(
    frame,
    listvariable=options_var,
    xscrollcommand=x_scroll.set,
    yscrollcommand=y_scroll.set,
)
grid(listbox, 0, 0)
x_scroll['command'] = listbox.xview
y_scroll['command'] = listbox.yview

root.mainloop()

## References

* [TkDocs Tutorial](https://tkdocs.com/tutorial/index.html)
* [Tkinter Reference](https://tkdocs.com/shipman/)